In [ ]:
# Python Clean Code

In [ ]:
# dataclass 활용


##############################################################################
class Point:
    def __init__(self, lat, long):
        self.lat = lat
        self.long = long


class Point:
    def __init__(self, lat: float, long: float) -> None:
        self.lat = lat
        self.long = long


##############################################################################
from dataclasses import dataclass


@dataclass
class Point:
    lat: float
    long: float

In [ ]:
# 적절한 타입 어노테이션_v1


##############################################################################
def launch_task(task, delay_in_seconds):
    pass


##############################################################################
Seconds = float


def launch_task(task, delay: Seconds):
    pass

In [ ]:
# 적절한 타입 어노테이션_v2


##############################################################################
def process_clients(clients: list):
    pass


##############################################################################
def process_clients(clients: list[tuple[int, str]]):
    pass


##############################################################################
from typing import Tuple

Client = Tuple[int, str]


def process_clients(clients: list[Client]):
    pass


##############################################################################
from dataclasses import dataclass


class Client:
    id: int
    name: str


def process_clients(clients: list[Client]):
    pass

In [ ]:
# __contains__의 활용


##############################################################################
def mark_coordinate(grid, coord):
    MARKED = 1
    if 0 <= coord.x < grid.width and 0 <= coord.y < grid.height:
        grid[coord] = MARKED


##############################################################################
class Boundaries:
    def __init__(self, width, height) -> None:
        self.width = width
        self.height = height

    def __contains__(self, coord):
        x, y = coord
        return 0 <= x < self.width and 0 <= y < self.height


class Grid:
    def __init__(self, width, height) -> None:
        self.width = width
        self.height = height
        self.boundaries = Boundaries(width, height)

    def __contains__(self, coord):
        return coord in self.boundaries


def mark_coordinate(grid, coord):
    MARKED = 1
    if coord in grid:
        grid[coord] = MARKED

In [3]:
# 코드 복잡성을 낮추기
from dataclasses import dataclass

USERS = [(i, f"first_name_{i}", f"last_name_{i}") for i in range(1_000)]


@dataclass
class User:
    user_id: int
    first_name: str
    last_name: str


##############################################################################
def bad_users_from_rows(dbrows) -> list:
    return [User(row[0], row[1], row[2]) for row in dbrows]


##############################################################################
def users_from_rows(dbrows) -> list:
    return [
        User(user_id, first_name, last_name)
        for (user_id, first_name, last_name) in dbrows
    ]

In [ ]:
# S : 단일 책임 원칙
class SystemMonitor:
    def load_activity(self):
        """소스에서 처리할 이벤트 가져오기"""

    def identify_events(self):
        """가저온 데이터를 파싱하여 도메인 객체 이벤트로 변환"""

    def stream_events(self):
        """파싱한 이벤트를 외부 에이전트로 전송"""


"""
독립적인 동작을 하는 메서드를 하나의 인터페이스에 정의함.
    각 동작이 나머지 부분과 독립적으로 수행할 수 있음
"""

In [ ]:
## O : 개방 / 폐쇄 원칙
"""
확장에는 개방되어 있으며, 
수정에는 폐쇄되어 있는 디자인
"""
from dataclasses import dataclass


################################################################
@dataclass
class Event:
    raw_data: dict


class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""


class LoginEvent(Event):
    """로그인 사용자에 의한 이벤트"""


class LogoutEvent(Event):
    """로그아웃 사용자에 의한 이벤트"""


class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data) -> None:
        self.event_data = event_data

    def identify_event(self):
        if (
            self.event_data["before"]["session"] == 0
            and self.event_data["after"]["session"] == 1
        ):
            return LoginEvent(self.event_data)

        elif (
            self.event_data["before"]["session"] == 1
            and self.event_data["after"]["session"] == 0
        ):
            return LogoutEvent(self.event_data)
        return UnknownEvent(self.event_data)


"""
문제 :
    이벤트 유형을 결정하는 로직이 단일 메서드에서 중앙 집중화 된다.
        지원하려는 이벤트가 늘어날수록 메서드도 커질 것이므로 결국 매우 큰 메서드가 될 것.
    메서드가 수정을 위해 닫히지 않았다.
        새로운 유형의 이벤트를 추가할 때마다 메서드를 수정해야한다.
그러므로
    1. 이 메서드를 변경하지 않고 새로운 유형의 이벤트를 추가하고 싶다 (폐쇄)
    2. 새로운 이벤트가 추가될 때 이미 존재하는 코드를 변경하지 않고 코드를 확장하여 지원하고 싶다 (개방)
"""

##############################################################################
@dataclass
class Event:
    raw_data: dict

    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return False


class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return (
            event_data["before"]["session"] == 0
            and event_data["after"]["session"] == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return (
            event_data["before"]["session"] == 1
            and event_data["after"]["session"] == 0
        )


class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data) -> None:
        self.event_data = event_data

    def identify_event(self):
        for event_cls in Event.__subclasses__():
            try:
                if event_cls.meets_condition(self.event_data):
                    return event_cls(self.event_data)
            except KeyError:
                continue
        return UnknownEvent(self.event_data)


## 추가시
class TransactionEvent(Event):
    """시스템에서 발생한 트랜잭션 이벤트"""

    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return event_data["after"].get("transaction") is not None

In [ ]:
# Interface를 통한 작업

from abc import abstractmethod, ABC


class XMLEventParser(ABC):
    @abstractmethod
    def from_xml(xml_data: str):
        """XML 형태 데이터 파싱"""


class JSONEventParser(ABC):
    @abstractmethod
    def from_json(json_data: str):
        """json 형태 데이터 파싱"""


class EventParser(XMLEventParser, JSONEventParser):
    def from_xml(xml_data: str):
        pass

    def from_json(json_data: str):
        pass